# AIF360 Inprocessing bias Mitigation with sklearn-compatible interface

In [1]:
%matplotlib inline
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from aif360.sklearn.postprocessing import RejectOptionClassifierCV, PostProcessingMeta
from aif360.sklearn.datasets import fetch_adult
from sklearn.metrics import accuracy_score
from aif360.sklearn.metrics import equal_opportunity_difference
from aif360.sklearn.postprocessing import CalibratedEqualizedOdds
from catboost import CatBoostClassifier, Pool
import numpy as np

`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df

## 1. Load data

In [2]:
X, y, sample_weight = fetch_adult()
print(X.head())

/Users/claudia/repositories/datascience-explained/venv/lib/python3.10/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


                 age  workclass     education  education-num  \
race      sex                                                  
Non-white Male  25.0    Private          11th            7.0   
White     Male  38.0    Private       HS-grad            9.0   
          Male  28.0  Local-gov    Assoc-acdm           12.0   
Non-white Male  44.0    Private  Some-college           10.0   
White     Male  34.0    Private          10th            6.0   

                    marital-status         occupation   relationship   race  \
race      sex                                                                 
Non-white Male       Never-married  Machine-op-inspct      Own-child  Black   
White     Male  Married-civ-spouse    Farming-fishing        Husband  White   
          Male  Married-civ-spouse    Protective-serv        Husband  White   
Non-white Male  Married-civ-spouse  Machine-op-inspct        Husband  Black   
White     Male       Never-married      Other-service  Not-in-family  White  

In [3]:
print(f"Dataset dimensions: {X.shape}")

Dataset dimensions: (45222, 13)


## 2. Preprocessing

Group multiple categories for race into white and non-white.

In [4]:
X.race = X.race.cat.set_categories(['Non-white', 'White'], ordered=True).fillna('Non-white')

Set index and label as integers.

In [5]:
X.index = pd.MultiIndex.from_arrays(X.index.codes, names=X.index.names)
y.index = pd.MultiIndex.from_arrays(y.index.codes, names=y.index.names)
y = pd.Series(y.factorize(sort=True)[0], index=y.index)

Split into train and test

In [6]:
(X_train, X_test,
 y_train, y_test) = train_test_split(X, y, train_size=0.7)

Transform categories into one hot encoding

In [7]:
ohe = make_column_transformer(
        (OneHotEncoder(sparse=False, handle_unknown="ignore"), X_train.dtypes == 'category'),
        remainder='passthrough', verbose_feature_names_out=False)
X_train  = pd.DataFrame(ohe.fit_transform(X_train), columns=ohe.get_feature_names_out(), index=X_train.index)
X_test = pd.DataFrame(ohe.transform(X_test), columns=ohe.get_feature_names_out(), index=X_test.index)

X_train.head()

/Users/claudia/repositories/datascience-explained/venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


workclass_Federal-gov  workclass_Local-gov  workclass_Private  \
race sex                                                                  
1    1                      0.0                  0.0                1.0   
     1                      0.0                  0.0                1.0   
     1                      0.0                  0.0                1.0   
     1                      0.0                  0.0                1.0   
     0                      0.0                  0.0                1.0   

          workclass_Self-emp-inc  workclass_Self-emp-not-inc  \
race sex                                                       
1    1                       0.0                         0.0   
     1                       0.0                         0.0   
     1                       0.0                         0.0   
     1                       0.0                         0.0   
     0                       0.0                         0.0   

          workclass_State-gov  workclass_Without-pay  education_10th  \
race sex                                                               
1    1                    0.0                    0.0             0.0   
     1                    0.0                    0.0             0.0   
     1                    0.0                    0.0             0.0   
     1                    0.0                    0.0             0.0   
     0                    0.0                    0.0             0.0   

          education_11th  education_12th  ...  native-country_Thailand  \
race sex                                  ...                            
1    1               0.0             0.0  ...                      0.0   
     1               0.0             0.0  ...                      0.0   
     1               0.0             0.0  ...                      0.0   
     1               0.0             0.0  ...                      0.0   
     0               0.0             0.0  ...                      0.0   

          native-country_Trinadad&Tobago  native-country_United-States  \
race sex                                                                 
1    1                               0.0                           1.0   
     1                               0.0                           1.0   
     1                               0.0                           1.0   
     1                               0.0                           1.0   
     0                               0.0                           1.0   

          native-country_Vietnam  native-country_Yugoslavia   age  \
race sex                                                            
1    1                       0.0                        0.0  58.0   
     1                       0.0                        0.0  45.0   
     1                       0.0                        0.0  33.0   
     1                       0.0                        0.0  30.0   
     0                       0.0                        0.0  47.0   

          education-num  capital-gain  capital-loss  hours-per-week  
race sex                                                             
1    1              9.0        3325.0           0.0            30.0  
     1              9.0           0.0           0.0            50.0  
     1              9.0           0.0           0.0            40.0  
     1             10.0           0.0           0.0            40.0  
     0              9.0           0.0           0.0            65.0  

[5 rows x 99 columns]

## 3. Baseline model

Train baseline logistic regression

In [8]:
baseline = LogisticRegression(solver='liblinear').fit(X_train, y_train)
y_pred = baseline.predict(X_test)
lr_acc = accuracy_score(y_test, y_pred)
lr_acc

0.8469079383798924

Compute equal opportunity difference.

In [9]:
lr_eod_sex = equal_opportunity_difference(y_test, y_pred, prot_attr='sex')
lr_eod_sex

-0.13331794676911995

In [10]:
lr_eod_race = equal_opportunity_difference(y_test, y_pred, prot_attr='race')
lr_eod_race

-0.11868153740074927

We see that it is negative, for sex and race, indicating that the true positive rate is higher for white and male.

## 4. Bias mitigation: RejectOptionClassifier postprocessing

In [11]:
metric = 'equal_opportunity'
roc_cv = PostProcessingMeta(baseline,
        RejectOptionClassifierCV('sex', scoring=metric, step=0.02, n_jobs=-1), prefit=True)
roc_cv.fit(X_train, y_train)
roc_cv_acc = accuracy_score(y_test, roc_cv.predict(X_test))
print(roc_cv_acc)

0.8074003095747033


/Users/claudia/repositories/datascience-explained/venv/lib/python3.10/site-packages/sklearn/utils/metaestimators.py:121: FutureWarning: if_delegate_has_method was deprecated in version 1.1 and will be removed in version 1.3. Use available_if instead.
  warnings.warn(


There is a 5% drop in accuracy

In [12]:
roc_cv_eod_sex = equal_opportunity_difference(y_test, roc_cv.predict(X_test), prot_attr='sex')
print(roc_cv_eod_sex)

# Check for improvement in average odds error for sex
assert abs(roc_cv_eod_sex)<abs(lr_eod_sex)

/Users/claudia/repositories/datascience-explained/venv/lib/python3.10/site-packages/sklearn/utils/metaestimators.py:121: FutureWarning: if_delegate_has_method was deprecated in version 1.1 and will be removed in version 1.3. Use available_if instead.
  warnings.warn(


-0.11677606381179573


We have mitigated up to some extent disparities for the sex variable.

## 5. Bias mitigation: CalibratedEqualizedOdds postprocessing

In [13]:
ceo = PostProcessingMeta(baseline,
        CalibratedEqualizedOdds('sex'), prefit=True)
ceo.fit(X_train, y_train)
ceo_acc = accuracy_score(y_test, ceo.predict(X_test))
print(ceo_acc)

0.8345986585096189


/Users/claudia/repositories/datascience-explained/venv/lib/python3.10/site-packages/sklearn/utils/metaestimators.py:121: FutureWarning: if_delegate_has_method was deprecated in version 1.1 and will be removed in version 1.3. Use available_if instead.
  warnings.warn(


There is a 5% drop in accuracy

In [14]:
ceo_eod_sex = equal_opportunity_difference(y_test, ceo.predict(X_test), prot_attr='sex')
print(ceo_eod_sex)

# Check for improvement in average odds error for sex
abs(ceo_eod_sex)<abs(lr_eod_sex)

-0.6170655567117586


/Users/claudia/repositories/datascience-explained/venv/lib/python3.10/site-packages/sklearn/utils/metaestimators.py:121: FutureWarning: if_delegate_has_method was deprecated in version 1.1 and will be removed in version 1.3. Use available_if instead.
  warnings.warn(


False

This method did not reduce the equal opportunity difference, our main goal.

## Bonus: Catboost classifier

We want to check if the postprocessing techniques are also working with more sophisticated ML models such as catboost.

In [15]:
(X_train, X_test,
 y_train, y_test) = train_test_split(X, y, train_size=0.7)
cat_vars = X_train.columns[np.where(X_train.dtypes == 'category')].tolist()
train_data = Pool(data=X_train, label=y_train, cat_features=cat_vars)
test_data = Pool(data=X_test, label=y_test, cat_features=cat_vars)

Train and evaluate basic catbooost classifier.

In [16]:
cbc = CatBoostClassifier(
        train_dir=None, silent=True, random_state=42
    )
cbc.fit(train_data, eval_set=[test_data], early_stopping_rounds=100, verbose_eval=100)
cb_acc = cbc.score(X_test, y_test)
y_pred = pd.Series(cbc.predict(X_test), index=y_test.index)
print(cb_acc)

Learning rate set to 0.074391
0:	learn: 0.6179988	test: 0.6194865	best: 0.6194865 (0)	total: 130ms	remaining: 2m 9s
100:	learn: 0.2860134	test: 0.2988849	best: 0.2988849 (100)	total: 8.84s	remaining: 1m 18s
200:	learn: 0.2728652	test: 0.2904323	best: 0.2904227 (199)	total: 14s	remaining: 55.7s
300:	learn: 0.2662003	test: 0.2882387	best: 0.2881841 (295)	total: 16.8s	remaining: 39s
400:	learn: 0.2600702	test: 0.2864908	best: 0.2864746 (399)	total: 23s	remaining: 34.4s
500:	learn: 0.2552426	test: 0.2859744	best: 0.2859706 (488)	total: 30.2s	remaining: 30.1s
600:	learn: 0.2508311	test: 0.2852974	best: 0.2852567 (576)	total: 34.3s	remaining: 22.8s
700:	learn: 0.2472747	test: 0.2852944	best: 0.2852023 (690)	total: 38.7s	remaining: 16.5s
800:	learn: 0.2436221	test: 0.2850722	best: 0.2849822 (770)	total: 44.8s	remaining: 11.1s
900:	learn: 0.2405652	test: 0.2850172	best: 0.2849079 (854)	total: 53.9s	remaining: 5.92s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.2849078829

In [17]:
cb_eod_sex = equal_opportunity_difference(y_test, y_pred, prot_attr='sex')
print(cb_eod_sex)
print(lr_eod_sex)
# Check for improvement in average odds error for sex
assert abs(cb_eod_sex)<abs(lr_eod_sex)

-0.08994227727886572
-0.13331794676911995


Apply mitigation strategy

In [18]:
roc_cbc = PostProcessingMeta(cbc,
        RejectOptionClassifierCV('sex', scoring=metric, step=0.02, n_jobs=-1), prefit=True)
roc_cbc.fit(X_train, y_train)
roc_cbc_acc = accuracy_score(y_test, roc_cbc.predict(X_test))
print(roc_cbc_acc)

0.8329770767303014


/Users/claudia/repositories/datascience-explained/venv/lib/python3.10/site-packages/sklearn/utils/metaestimators.py:121: FutureWarning: if_delegate_has_method was deprecated in version 1.1 and will be removed in version 1.3. Use available_if instead.
  warnings.warn(


In [19]:
roc_cbc_eod_sex = equal_opportunity_difference(y_test, roc_cbc.predict(X_test), prot_attr='sex')
print(roc_cbc_eod_sex)

# Check for improvement in average odds error for sex
abs(roc_cbc_eod_sex)<abs(lr_eod_sex)

-0.06952420915091695


/Users/claudia/repositories/datascience-explained/venv/lib/python3.10/site-packages/sklearn/utils/metaestimators.py:121: FutureWarning: if_delegate_has_method was deprecated in version 1.1 and will be removed in version 1.3. Use available_if instead.
  warnings.warn(


True